In [8]:
# Ends with most recent data in editor month table
START = "2017-06-01"

# Edits

In [9]:
edits = run_mariadb(
    """
    select 
    month, 
    sum(edits) as non_bot_edits,
    sum(if(user_name = "", edits, 0)) as anon_edits,
    sum(mobile_web_edits) as mobile_web_edits,
    sum(mobile_app_edits) as mobile_app_edits
    from (
    select month, user_name, sum(edits) as edits,
    sum(mobile_web_edits) as mobile_web_edits,
    sum(mobile_app_edits) as mobile_app_edits,
    max(bot_flag) as bot_flag
    from staging.editor_month
    where month >= "{start}"
    group by month, user_name
    ) global_edits
    where
    bot_flag = 0 and
    convert(user_name using utf8) not regexp "bot\\b" and
    convert(user_name using utf8) not regexp "[ _]bot[ _]"
    group by month;
    """.format(start = START)
)

/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pymysql/cursors.py:323: Warning: (1300, "Invalid utf8 character string: 'F09D92'")
  self._do_get_result()
/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pymysql/cursors.py:323: Warning: (1300, "Invalid utf8 character string: 'F09D95'")
  self._do_get_result()
/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pymysql/cursors.py:323: Warning: (1300, "Invalid utf8 character string: 'F09F92'")
  self._do_get_result()
/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pymysql/cursors.py:323: Warning: (1300, "Invalid utf8 character string: 'F09F98'")
  self._do_get_result()
/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pymysql/cursors.py:323: Warning: (1300, "Invalid utf8 character string: 'F09F8C'")
  self._do_get_result()
/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pymysql/cursors.py:323: Warning: (1300, "Invalid utf8 character string: 'F0908C'")
  self._do_get_result()
/home/neilpquinn-wmf/venv/lib/pyth

In [10]:
edits

,month,non_bot_edits,anon_edits,mobile_web_edits,mobile_app_edits
0,2017-06-01,19133345.0,2044585.0,762855.0,91197.0
1,2017-07-01,20947385.0,2065358.0,827436.0,105578.0
2,2017-08-01,23447716.0,2100485.0,900747.0,106198.0
3,2017-09-01,20236462.0,2043426.0,828826.0,94536.0
4,2017-10-01,22513844.0,2211979.0,897582.0,100701.0
5,2017-11-01,23101935.0,2157859.0,892819.0,94125.0
6,2017-12-01,19926661.0,2087239.0,924216.0,96437.0


# Editors

In [11]:
editors = run_mariadb(
    """
    select 
    month, 
    count(*) as active_editors,
    sum(extract(year_month from reg) = extract(year_month from month)) as new_active_editors,
    sum(extract(year_month from reg) = extract(year_month from date_sub(month, interval 1 month))) as second_month_active_editors
    from (
    select month, user_name, sum(content_edits) as content_edits, 
    min(user_registration) as reg, max(bot_flag) as bot_flag
    from staging.editor_month
    where local_user_id != 0 and month >= "{start}"
    group by month, user_name
    ) global_edits
    where 
    content_edits >= 5 and bot_flag = 0 and
    convert(user_name using utf8) not regexp "bot\\b" and
    convert(user_name using utf8) not regexp "[ _]bot[ _]"
    group by month;
    """.format(start = START)
)

/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pymysql/cursors.py:323: Warning: (1300, "Invalid utf8 character string: 'F09D92'")
  self._do_get_result()
/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pymysql/cursors.py:323: Warning: (1300, "Invalid utf8 character string: 'F09F98'")
  self._do_get_result()
/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pymysql/cursors.py:323: Warning: (1300, "Invalid utf8 character string: 'F0908C'")
  self._do_get_result()
/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pymysql/cursors.py:323: Warning: (1300, "Invalid utf8 character string: 'F09D94'")
  self._do_get_result()


In [12]:
editors

,month,active_editors,new_active_editors,second_month_active_editors
0,2017-06-01,81689,18771.0,4112.0
1,2017-07-01,80170,16383.0,4055.0
2,2017-08-01,79204,15316.0,3780.0
3,2017-09-01,79635,16909.0,3637.0
4,2017-10-01,81088,16518.0,3998.0
5,2017-11-01,82770,16833.0,4572.0
6,2017-12-01,79580,14393.0,4152.0
